# HLW-COVID Brasil - Análise de Resultados

Este notebook analisa os resultados da estimação do modelo HLW-COVID:
- Taxa natural de juros (r*)
- PIB potencial (y*)
- Hiato do produto (ỹ)
- Crescimento tendencial (g)
- Hiato da taxa real (r - r*)

## 1. Setup e Importações

In [ ]:
import sys
from pathlib import Path
import pickle

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

print("✓ Imports successful")

## 2. Carregar Resultados

In [ ]:
# Load results from CSV
results_df = pd.read_csv(project_root / "outputs" / "results" / "hlw_estimates.csv")
results_df['date'] = pd.to_datetime(results_df['date'])

# Load parameters
params_df = pd.read_csv(project_root / "outputs" / "results" / "parameter_estimates.csv")

# Load full results object
with open(project_root / "outputs" / "results" / "estimation_results.pkl", 'rb') as f:
    full_results = pickle.load(f)

print(f"Dados carregados:")
print(f"  Período: {results_df['date'].min()} a {results_df['date'].max()}")
print(f"  Observações: {len(results_df)}")
print(f"\nColunas: {results_df.columns.tolist()}")

## 3. Parâmetros Estimados

In [ ]:
print("=" * 60)
print("PARÂMETROS ESTIMADOS")
print("=" * 60)
display(params_df)

# Calculate t-statistics
params_df['t_stat'] = params_df['estimate'] / params_df['std_error']
params_df['significant'] = abs(params_df['t_stat']) > 1.96

print("\nSignificância estatística (|t| > 1.96):")
display(params_df[['parameter', 'estimate', 'std_error', 't_stat', 'significant']])

## 4. Taxa Natural de Juros (r*)

In [ ]:
# Plot r* with confidence bands
fig, ax = plt.subplots(figsize=(14, 6))

# r*
ax.plot(results_df['date'], results_df['r_star'], 
        linewidth=2.5, label='r* (taxa natural)', color='darkblue')

# Confidence bands (±1.96 SE)
ax.fill_between(
    results_df['date'],
    results_df['r_star'] - 1.96 * results_df['r_star_se'],
    results_df['r_star'] + 1.96 * results_df['r_star_se'],
    alpha=0.3,
    label='IC 95%'
)

# Actual real rate
ax.plot(results_df['date'], results_df['r_t'], 
        linewidth=1.5, label='r_t (taxa real observada)', 
        color='orange', alpha=0.7, linestyle='--')

# COVID period
covid_start = pd.Timestamp('2020-04-01')
covid_end = pd.Timestamp('2022-12-31')
ax.axvspan(covid_start, covid_end, alpha=0.1, color='red', label='Período COVID')

ax.set_title('Taxa Natural de Juros (r*) - Brasil', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Taxa (% a.a.)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "r_star_series.png", dpi=300, bbox_inches='tight')
plt.show()

# Summary statistics
print("\nEstatísticas de r*:")
print(results_df['r_star'].describe())

## 5. Hiato da Taxa Real (r - r*)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Real rate gap
ax.plot(results_df['date'], results_df['r_gap'], 
        linewidth=2.5, color='darkgreen', label='Hiato da taxa real (r - r*)')

# Zero line
ax.axhline(y=0, color='black', linestyle='--', linewidth=1.5, label='Neutralidade')

# Shade positive/negative regions
ax.fill_between(results_df['date'], 0, results_df['r_gap'], 
                where=(results_df['r_gap'] >= 0), 
                alpha=0.3, color='red', label='Política contracionista')
ax.fill_between(results_df['date'], 0, results_df['r_gap'], 
                where=(results_df['r_gap'] < 0), 
                alpha=0.3, color='blue', label='Política expansionista')

# COVID period
ax.axvspan(covid_start, covid_end, alpha=0.1, color='gray')

ax.set_title('Hiato da Taxa Real de Juros (r - r*)', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Pontos percentuais', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "real_rate_gap.png", dpi=300, bbox_inches='tight')
plt.show()

## 6. PIB Observado vs Potencial

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Observed and potential GDP
ax.plot(results_df['date'], results_df['y_t'], 
        linewidth=2, label='PIB observado (y_t)', color='darkblue')
ax.plot(results_df['date'], results_df['y_star'], 
        linewidth=2, label='PIB potencial (y*)', color='red', linestyle='--')

# COVID period
ax.axvspan(covid_start, covid_end, alpha=0.1, color='gray', label='Período COVID')

ax.set_title('PIB Real: Observado vs Potencial', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Log(PIB)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "gdp_actual_vs_potential.png", dpi=300, bbox_inches='tight')
plt.show()

## 7. Hiato do Produto

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Output gap
ax.plot(results_df['date'], results_df['y_gap'] * 100, 
        linewidth=2.5, color='purple', label='Hiato do produto (ỹ)')

# Zero line
ax.axhline(y=0, color='black', linestyle='--', linewidth=1.5)

# Shade positive/negative regions
ax.fill_between(results_df['date'], 0, results_df['y_gap'] * 100, 
                where=(results_df['y_gap'] >= 0), 
                alpha=0.3, color='green', label='Economia aquecida')
ax.fill_between(results_df['date'], 0, results_df['y_gap'] * 100, 
                where=(results_df['y_gap'] < 0), 
                alpha=0.3, color='red', label='Economia abaixo do potencial')

# COVID period
ax.axvspan(covid_start, covid_end, alpha=0.1, color='gray')

ax.set_title('Hiato do Produto (% do PIB potencial)', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Hiato (%)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "output_gap.png", dpi=300, bbox_inches='tight')
plt.show()

## 8. Crescimento Tendencial (g)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Trend growth (convert to annualized %)
g_annualized = results_df['g'] * 400  # Quarterly to annual percentage

ax.plot(results_df['date'], g_annualized, 
        linewidth=2.5, color='darkgreen', label='Crescimento tendencial (g)')

# COVID period
ax.axvspan(covid_start, covid_end, alpha=0.1, color='gray', label='Período COVID')

ax.set_title('Taxa de Crescimento Tendencial do PIB', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Taxa (% a.a.)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "trend_growth.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\nCrescimento tendencial médio: {g_annualized.mean():.2f}% a.a.")
print(f"Pré-COVID: {g_annualized[results_df['date'] < covid_start].mean():.2f}% a.a.")
print(f"Pós-COVID: {g_annualized[results_df['date'] > covid_end].mean():.2f}% a.a.")

## 9. Dashboard Completo

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(18, 14))

# 1. r*
axes[0, 0].plot(results_df['date'], results_df['r_star'], linewidth=2, color='darkblue')
axes[0, 0].fill_between(
    results_df['date'],
    results_df['r_star'] - 1.96 * results_df['r_star_se'],
    results_df['r_star'] + 1.96 * results_df['r_star_se'],
    alpha=0.3
)
axes[0, 0].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[0, 0].set_title('Taxa Natural de Juros (r*)', fontweight='bold')
axes[0, 0].set_ylabel('% a.a.')
axes[0, 0].grid(True, alpha=0.3)

# 2. r - r*
axes[0, 1].plot(results_df['date'], results_df['r_gap'], linewidth=2, color='darkgreen')
axes[0, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[0, 1].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[0, 1].set_title('Hiato da Taxa Real (r - r*)', fontweight='bold')
axes[0, 1].set_ylabel('p.p.')
axes[0, 1].grid(True, alpha=0.3)

# 3. y_t vs y*
axes[1, 0].plot(results_df['date'], results_df['y_t'], linewidth=2, label='Observado')
axes[1, 0].plot(results_df['date'], results_df['y_star'], linewidth=2, 
               linestyle='--', label='Potencial')
axes[1, 0].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[1, 0].set_title('PIB: Observado vs Potencial', fontweight='bold')
axes[1, 0].set_ylabel('Log(PIB)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Output gap
axes[1, 1].plot(results_df['date'], results_df['y_gap'] * 100, 
               linewidth=2, color='purple')
axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1, 1].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[1, 1].set_title('Hiato do Produto', fontweight='bold')
axes[1, 1].set_ylabel('%')
axes[1, 1].grid(True, alpha=0.3)

# 5. Trend growth
axes[2, 0].plot(results_df['date'], results_df['g'] * 400, 
               linewidth=2, color='darkgreen')
axes[2, 0].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[2, 0].set_title('Crescimento Tendencial', fontweight='bold')
axes[2, 0].set_ylabel('% a.a.')
axes[2, 0].grid(True, alpha=0.3)

# 6. Inflation
axes[2, 1].plot(results_df['date'], results_df['pi_t'], 
               linewidth=2, color='orange')
axes[2, 1].axvspan(covid_start, covid_end, alpha=0.1, color='gray')
axes[2, 1].set_title('Inflação (π_t)', fontweight='bold')
axes[2, 1].set_ylabel('% a.a.')
axes[2, 1].grid(True, alpha=0.3)

plt.suptitle('HLW-COVID Brasil - Dashboard de Resultados', 
             fontsize=18, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(project_root / "outputs" / "figures" / "dashboard_completo.png", 
            dpi=300, bbox_inches='tight')
plt.show()

## 10. Análise do Período COVID

In [ ]:
# Filter COVID period
covid_mask = (results_df['date'] >= covid_start) & (results_df['date'] <= covid_end)
covid_results = results_df[covid_mask]

print("=" * 60)
print("ANÁLISE DO PERÍODO COVID (2020-Q2 a 2022-Q4)")
print("=" * 60)

print(f"\nr* médio durante COVID: {covid_results['r_star'].mean():.4f}")
print(f"r* pré-COVID: {results_df[~covid_mask & (results_df['date'] < covid_start)]['r_star'].mean():.4f}")
print(f"Variação: {covid_results['r_star'].mean() - results_df[~covid_mask & (results_df['date'] < covid_start)]['r_star'].mean():.4f}")

print(f"\nHiato do produto mínimo: {covid_results['y_gap'].min() * 100:.2f}%")
print(f"Recuperação até fim do período: {covid_results['y_gap'].iloc[-1] * 100:.2f}%")

# COVID effects from parameters
kappa_2020 = params_df[params_df['parameter'] == 'kappa_2020']['estimate'].values[0]
kappa_2021 = params_df[params_df['parameter'] == 'kappa_2021']['estimate'].values[0]
kappa_2022 = params_df[params_df['parameter'] == 'kappa_2022']['estimate'].values[0]

print(f"\nEfeitos COVID na inflação (κ):")
print(f"  2020: {kappa_2020:.4f}")
print(f"  2021: {kappa_2021:.4f}")
print(f"  2022: {kappa_2022:.4f}")

beta = params_df[params_df['parameter'] == 'beta']['estimate'].values[0]
print(f"\nEfeito do Stringency Index em r* (β): {beta:.6f}")

## 11. Comparação com Literatura

In [ ]:
# Get recent r* estimates
recent_r_star = results_df['r_star'].iloc[-4:].mean()  # Last 4 quarters

print("=" * 60)
print("COMPARAÇÃO COM LITERATURA")
print("=" * 60)

print(f"\nr* estimado (média últimos 4 trimestres): {recent_r_star:.2f}% a.a.")
print(f"\nReferências para comparação:")
print(f"  - EUA (Holston et al., 2017): ~0.5% a 1.0% a.a.")
print(f"  - Brasil (estimativas prévias): Variável, mas tipicamente > 3%")
print(f"\nObs: Diferenças podem refletir:")
print(f"  1. Especificação do modelo (ajustes COVID)")
print(f"  2. Período amostral")
print(f"  3. Dados utilizados")
print(f"  4. Prêmio de risco país")

## 12. Exportar Tabelas para LaTeX

In [ ]:
# Export parameter table to LaTeX
params_latex = params_df[['parameter', 'estimate', 'std_error', 't_stat']].copy()
params_latex.columns = ['Parâmetro', 'Estimativa', 'Erro Padrão', 't-stat']

latex_table = params_latex.to_latex(
    index=False,
    float_format="%.4f",
    caption="Parâmetros Estimados do Modelo HLW-COVID",
    label="tab:hlw_params"
)

with open(project_root / "outputs" / "results" / "parameters_table.tex", 'w') as f:
    f.write(latex_table)

print("✓ Tabela LaTeX exportada para: outputs/results/parameters_table.tex")

## 13. Resumo Final

In [ ]:
print("=" * 80)
print("RESUMO EXECUTIVO - HLW-COVID BRASIL")
print("=" * 80)

print(f"\n1. TAXA NATURAL DE JUROS (r*)")
print(f"   Estimativa atual: {results_df['r_star'].iloc[-1]:.2f}% a.a.")
print(f"   IC 95%: [{results_df['r_star'].iloc[-1] - 1.96*results_df['r_star_se'].iloc[-1]:.2f}%, "
      f"{results_df['r_star'].iloc[-1] + 1.96*results_df['r_star_se'].iloc[-1]:.2f}%]")
print(f"   Média histórica: {results_df['r_star'].mean():.2f}% a.a.")

print(f"\n2. ORIENTAÇÃO DA POLÍTICA MONETÁRIA")
current_gap = results_df['r_gap'].iloc[-1]
if current_gap > 0.5:
    orientation = "CONTRACIONISTA"
elif current_gap < -0.5:
    orientation = "EXPANSIONISTA"
else:
    orientation = "NEUTRA"
print(f"   Hiato atual (r - r*): {current_gap:.2f} p.p.")
print(f"   Orientação: {orientation}")

print(f"\n3. HIATO DO PRODUTO")
print(f"   Hiato atual: {results_df['y_gap'].iloc[-1]*100:.2f}%")
if results_df['y_gap'].iloc[-1] > 0:
    print(f"   Economia acima do potencial (pressão inflacionária)")
else:
    print(f"   Economia abaixo do potencial (capacidade ociosa)")

print(f"\n4. CRESCIMENTO TENDENCIAL")
print(f"   Taxa atual: {results_df['g'].iloc[-1]*400:.2f}% a.a.")
print(f"   Média pós-2015: {results_df[results_df['date'] >= '2015-01-01']['g'].mean()*400:.2f}% a.a.")

print(f"\n5. EFEITOS COVID")
print(f"   κ_2020 = {kappa_2020:.4f} ({kappa_2020*100:.2f} p.p. na inflação)")
print(f"   κ_2021 = {kappa_2021:.4f} ({kappa_2021*100:.2f} p.p. na inflação)")
print(f"   κ_2022 = {kappa_2022:.4f} ({kappa_2022*100:.2f} p.p. na inflação)")
print(f"   β (stringency → r*) = {beta:.6f}")

print("\n" + "=" * 80)
print("FIM DA ANÁLISE")
print("=" * 80)